In [1]:
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# 1. Data loading & preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),  # converts to [0,1] tensor
    transforms.Normalize((0.5,), (0.5,))  # normalize
])

train_dataset = datasets.MNIST(root="./data", train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root="./data", train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# 2. Define CNN (similar to Keras Sequential model)
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=0)
        self.conv2 = nn.Conv2d(32, 32, 3, padding=0)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout1 = nn.Dropout(0.25)
        self.fc1 = nn.Linear(32 * 12 * 12, 128)  # after pooling & convs
        self.dropout2 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = self.dropout1(x)
        x = x.view(x.size(0), -1)  # flatten
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

model = CNN()

# 3. Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 4. Training loop
for epoch in range(1):  # like nb_epoch=1
    model.train()
    for data, target in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1} complete, loss={loss.item():.4f}")

# 5. Testing predictions (like your random 10 samples)
model.eval()
examples = np.random.choice(len(test_dataset), size=10, replace=False)

for idx in examples:
    img, label = test_dataset[idx]
    with torch.no_grad():
        output = model(img.unsqueeze(0))  # add batch dim
        pred = output.argmax(dim=1).item()
    
    image = img.squeeze().numpy() * 255
    image = image.astype("uint8")

    print(f"Actual digit is {label}, predicted {pred}")
    
    # Show image for 500ms (half a second), then close
    cv2.imshow("Digit", image)
    cv2.waitKey(500)  
    cv2.destroyAllWindows()


Epoch 1 complete, loss=0.2134
Actual digit is 9, predicted 9
Actual digit is 1, predicted 1
Actual digit is 0, predicted 0
Actual digit is 4, predicted 4
Actual digit is 7, predicted 7
Actual digit is 9, predicted 9
Actual digit is 2, predicted 2
Actual digit is 2, predicted 2
Actual digit is 1, predicted 1
Actual digit is 9, predicted 9
